В этом обзоре последняя https://github.com/deepinsight/insightface

Забираем 2d106det.zip из https://github.com/deepinsight/insightface/tree/master/alignment/coordinateReg

https://drive.google.com/file/d/1MBWbTEYRhZFzj_O2f2Dc6fWGXFWtbMFw/view?usp=sharing

In [1]:
!pip install cmake

In [ ]:
#conda install mxnet

In [ ]:
!pip install mxnet

  Using cached mxnet-1.6.0-py2.py3-none-win_amd64.whl (26.9 MB)
  Using cached requests-2.18.4-py2.py3-none-any.whl (88 kB)
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Using cached idna-2.6-py2.py3-none-any.whl (56 kB)
  Using cached urllib3-1.22-py2.py3-none-any.whl (132 kB)
  Attempting uninstall: idna
    Found existing installation: idna 2.8
    Uninstalling idna-2.8:
      Successfully uninstalled idna-2.8
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.8
    Uninstalling urllib3-1.25.8:
      Successfully uninstalled urllib3-1.25.8
  Attempting uninstall: requests
    Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0


In [ ]:
!pip install insightface

  Created wheel for insightface: filename=insightface-0.1.5-py3-none-any.whl size=16768 sha256=09e2c558936546778437a2d7bb131a9c65f41e6d1458db7456ceac244883a888
  Stored in directory: c:\users\lex\appdata\local\pip\cache\wheels\84\99\f5\83f86bd43d6d03d19809d577cebcf3548a2ac662439472b5e0
  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6354 sha256=02a29d3f0c3182cee2f425cbc6a896c31bb86f71fa47ea8da28598dde4a17fac
  Stored in directory: c:\users\lex\appdata\local\pip\cache\wheels\88\96\68\c2be18e7406804be2e593e1c37845f2dd20ac2ce1381ce40b0
Successfully built insightface easydict


In [ ]:
!pip install opencv-python

In [ ]:
from zipfile import ZipFile
with ZipFile('./2d106det.zip', 'r') as zf:
    zf.extractall('./')

In [ ]:
!unzip './2d106det.zip'

"unzip" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


In [ ]:
import argparse
import cv2
import sys
import numpy as np
import os
import mxnet as mx
import datetime
from skimage import transform as trans
import insightface

In [ ]:
def square_crop(im, S):
  if im.shape[0]>im.shape[1]:
    height = S
    width = int( float(im.shape[1]) / im.shape[0] * S )
    scale = float(S) / im.shape[0]
  else:
    width = S
    height = int( float(im.shape[0]) / im.shape[1] * S )
    scale = float(S) / im.shape[1]
  resized_im = cv2.resize(im, (width, height))
  det_im = np.zeros( (S, S, 3), dtype=np.uint8 )
  det_im[:resized_im.shape[0], :resized_im.shape[1], :] = resized_im
  return det_im, scale

In [ ]:
def transform(data, center, output_size, scale, rotation):
    scale_ratio = scale
    rot = float(rotation)*np.pi/180.0
    #translation = (output_size/2-center[0]*scale_ratio, output_size/2-center[1]*scale_ratio)
    t1 = trans.SimilarityTransform(scale=scale_ratio)
    cx = center[0]*scale_ratio
    cy = center[1]*scale_ratio
    t2 = trans.SimilarityTransform(translation=(-1*cx, -1*cy))
    t3 = trans.SimilarityTransform(rotation=rot)
    t4 = trans.SimilarityTransform(translation=(output_size/2, output_size/2))
    t = t1+t2+t3+t4
    M = t.params[0:2]
    cropped = cv2.warpAffine(data,M,(output_size, output_size), borderValue = 0.0)
    return cropped, M

def trans_points2d(pts, M):
    new_pts = np.zeros(shape=pts.shape, dtype=np.float32)
    for i in range(pts.shape[0]):
        pt = pts[i]
        new_pt = np.array([pt[0], pt[1], 1.], dtype=np.float32)
        new_pt = np.dot(M, new_pt)
        #print('new_pt', new_pt.shape, new_pt)
        new_pts[i] = new_pt[0:2]

    return new_pts

def trans_points3d(pts, M):
    scale = np.sqrt(M[0][0]*M[0][0] + M[0][1]*M[0][1])
    #print(scale)
    new_pts = np.zeros(shape=pts.shape, dtype=np.float32)
    for i in range(pts.shape[0]):
        pt = pts[i]
        new_pt = np.array([pt[0], pt[1], 1.], dtype=np.float32)
        new_pt = np.dot(M, new_pt)
        #print('new_pt', new_pt.shape, new_pt)
        new_pts[i][0:2] = new_pt[0:2]
        new_pts[i][2] = pts[i][2]*scale

    return new_pts

def trans_points(pts, M):
  if pts.shape[1]==2:
    return trans_points2d(pts, M)
  else:
    return trans_points3d(pts, M)

In [ ]:
class Handler:
  def __init__(self, prefix, epoch, im_size=192, det_size=224, ctx_id=0):
    print('loading',prefix, epoch)
    if ctx_id>=0:
      ctx = mx.gpu(ctx_id)
    else:
      ctx = mx.cpu()
    image_size = (im_size, im_size)
    self.detector = insightface.model_zoo.get_model('retinaface_mnet025_v2') #can replace with your own face detector
    #self.detector = insightface.model_zoo.get_model('retinaface_r50_v1')
    self.detector.prepare(ctx_id=ctx_id)
    self.det_size = det_size
    sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)
    all_layers = sym.get_internals()
    sym = all_layers['fc1_output']
    self.image_size = image_size
    model = mx.mod.Module(symbol=sym, context=ctx, label_names = None)
    model.bind(for_training=False, data_shapes=[('data', (1, 3, image_size[0], image_size[1]))])
    model.set_params(arg_params, aux_params)
    self.model = model
    self.image_size = image_size

  def get(self, img, get_all=False):
    out = []
    det_im, det_scale = square_crop(img, self.det_size)
    bboxes, _ = self.detector.detect(det_im)
    if bboxes.shape[0]==0:
        return out
    bboxes /= det_scale
    if not get_all:
      areas = []
      for i in range(bboxes.shape[0]):
        x = bboxes[i]
        area = (x[2]-x[0])*(x[3]-x[1])
        areas.append(area)
      m = np.argsort(areas)[-1]
      bboxes = bboxes[m:m+1]
    for i in range(bboxes.shape[0]):
      bbox = bboxes[i]
      input_blob = np.zeros( (1, 3)+self.image_size,dtype=np.float32)
      w, h = (bbox[2]-bbox[0]), (bbox[3]-bbox[1])
      center = (bbox[2]+bbox[0])/2, (bbox[3]+bbox[1])/2
      rotate = 0
      _scale = self.image_size[0]*2/3.0/max(w,h)
      rimg, M = transform(img, center, self.image_size[0], _scale, rotate)
      rimg = cv2.cvtColor(rimg, cv2.COLOR_BGR2RGB)
      rimg = np.transpose(rimg, (2,0,1)) #3*112*112, RGB
      input_blob[0] = rimg
      data = mx.nd.array(input_blob)
      db = mx.io.DataBatch(data=(data,))
      self.model.forward(db, is_train=False)
      pred = self.model.get_outputs()[-1].asnumpy()[0]
      if pred.shape[0]>=3000:
        pred = pred.reshape( (-1, 3) )
      else:
        pred = pred.reshape( (-1, 2) )
      pred[:,0:2] += 1
      pred[:,0:2] *= (self.image_size[0]//2)
      if pred.shape[1]==3:
        pred[:,2] *= (self.image_size[0]//2)

      IM = cv2.invertAffineTransform(M)
      pred = trans_points(pred, IM)
      out.append(pred)
    return out    

In [ ]:
#if __name__ == '__main__':
handler = Handler('./2d106det', 0, ctx_id=-1, det_size=640)
im = cv2.imread('./t1.jpg')
tim = im.copy()
preds = handler.get(im, get_all=True)
color = (200, 160, 75)
for pred in preds:
    pred = np.round(pred).astype(np.int)
    for i in range(pred.shape[0]):
      p = tuple(pred[i])
      cv2.circle(tim, p, 1, color, 1,cv2.LINE_AA)
cv2.imwrite('./out1.jpg', tim)

loading ./2d106det 0
Model file is not found. Downloading.


1825KB [00:03, 521.46KB/s]                          


[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


True

In [ ]:
!pip install git+https://github.com/letmaik/pyvirtualcam

  Cloning https://github.com/letmaik/pyvirtualcam to c:\users\lex\appdata\local\temp\pip-req-build-ogkfnu7e
  Created wheel for pyvirtualcam: filename=pyvirtualcam-0.2.0-cp37-cp37m-win_amd64.whl size=59647 sha256=d72f8caf13faff80cbfc151548d5796c57ac3cb8882b67522bfd4c9594ea8e65
  Stored in directory: C:\Users\lex\AppData\Local\Temp\pip-ephem-wheel-cache-f1w86m4d\wheels\bb\8b\67\a412a1cbed4f302725eb610ef24ad5dd7d38d34c35eefa0494
Successfully built pyvirtualcam


  Running command git clone -q https://github.com/letmaik/pyvirtualcam 'C:\Users\lex\AppData\Local\Temp\pip-req-build-ogkfnu7e'


In [ ]:
import pyvirtualcam
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFilter

In [ ]:
cap = cv2.VideoCapture(0)
#img_anime = Image.open('./t2.jpg')

https://github.com/CatxFish/obs-virtual-cam/releases
https://github.com/johnboiles/obs-mac-virtualcam
https://github.com/letmaik/pyvirtualcam

In [ ]:
with pyvirtualcam.Camera(width=640, height=480, fps=30) as cam:    
    while True:
        _,frame=cap.read()
        
        tim = frame.copy()
        preds = handler.get(frame, get_all=True)
        color = (200, 160, 75)
        for pred in preds:
            pred = np.round(pred).astype(np.int)
            for i in range(pred.shape[0]):
              p = tuple(pred[i])
              cv2.circle(tim, p, 1, color, 1,cv2.LINE_AA)
        
        tim = cv2.cvtColor(tim, cv2.COLOR_RGB2RGBA)            
        cam.send(tim) 
        cv2.waitKey(1)
        cam.sleep_until_next_frame()

RuntimeError: error starting virtual camera output

In [ ]:
with pyvirtualcam.Camera(width=1280, height=720, fps=30) as cam:
    while True:
        frame = np.zeros((cam.height, cam.width, 4), np.uint8) # RGBA
        frame[:,:,:3] = cam.frames_sent % 255 # grayscale animation
        frame[:,:,3] = 255
        cam.send(frame)
        cam.sleep_until_next_frame()

RuntimeError: error starting virtual camera output